In [1]:
import pymatgen


In [2]:
import ase.db

In [3]:
import numpy as np

In [4]:
import collections as coll

In [5]:
!wget https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db

--2017-06-15 22:02:27--  https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db
Resolving cmr.fysik.dtu.dk (cmr.fysik.dtu.dk)... 130.225.86.24, 2001:878:200:2010:3::24
Connecting to cmr.fysik.dtu.dk (cmr.fysik.dtu.dk)|130.225.86.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4542464 (4.3M) [text/plain]
Saving to: ‘mp_gllbsc.db.6’

mp_gllbsc.db.6      100%[=====================>]   4.33M  1.30MB/s   in 4.2s   

2017-06-15 22:02:32 (1.03 MB/s) - ‘mp_gllbsc.db.6’ saved [4542464/4542464]



In [6]:
con = ase.db.connect('mp_gllbsc.db')

In [7]:
from mpcontribs.io.archieml.mpfile import MPFile  

In [8]:
s = """
title: GLLB-SC Bandgaps
references.AEM: Advanced Energy Materials, Juli 22, 2014
references.PRA: O. Gritsenko, R. van  Leeuwen, E. van Lenthe, E. J. Baerends, Phys. Rev. A 1995, 51, 1944.  
references.PRB: M. Kuisma, J. Ojanen, J. Enkovaara, T. T. Rantala, Phys. Rev. B 2010, 82, 115106.  

contributor : Technical University of Denmark
authors: Ivano E. Castelli, Falco Hueser, Mohnish Pandey, Hong Li, Kristian S. Thygesen, Brian Seger, Anubhav Jain, Kristin A. Persson, Gerbrand Ceder, and Karsten W. Jacobsen   
explanation: Bandgaps calculated using GLLB-SC potential by Gritsenko, van Leeuwen, van Lenthe, and Baerends (GLLB), adapted by Kuisma et al to account for solids (-SC). The Kohn-Sham gap most basically refers to the energy difference between the bottom of the conduction band and the top of the valence band. The Quasi-Particle gap takes into account the derivative discontinuity of the chemical potential and gives the bandgap as the difference between ionization potenital and electron affinity. 

"""


In [9]:
mpfile = MPFile.from_string(s)
count = 0
for row in con.select('mpid'):
    d = coll.OrderedDict([])
    d['Kohn-Sham_Bandgap'] = coll.OrderedDict([])
    d['Derivative_Discontinuity'] = coll.OrderedDict([]) 
    d['Quasi-Particle_Bandgap'] = coll.OrderedDict([])
    count = count + 1
    mpid = 'mp-' + str(row.mpid)
    d['Kohn-Sham_Bandgap']['Indirect'] = row.gllbsc_ind_gap - row.gllbsc_disc
    d['Kohn-Sham_Bandgap']['Direct'] = row.gllbsc_dir_gap - row.gllbsc_disc
    d['Derivative_Discontinuity'] = row.gllbsc_disc
    d['Quasi-Particle_Bandgap']['Indirect'] = row.gllbsc_ind_gap
    d['Quasi-Particle_Bandgap']['Direct'] = row.gllbsc_dir_gap
    mpfile.add_hierarchical_data(mpid,d) 
    if count == 10:
        break
        
     
       
 



In [81]:
import pandas as pd
data1 = np.loadtxt('dtu_gllbsc_gaps.txt')
data_txt = pd.DataFrame(data1)
data_txt = data_txt.sort_values(0)
data = []
for row in con.select('mpid'):
    data.append([row.mpid, row.gllbsc_ind_gap - row.gllbsc_disc, row.gllbsc_dir_gap - row.gllbsc_disc, row.gllbsc_disc,
                row.gllbsc_ind_gap, row.gllbsc_dir_gap])


  
data_db = pd.DataFrame(data)
data_db = data_db.sort_values(0)


data_txt['6'] = 'txt'
data_db['6'] = 'db'
data_all = pd.concat([data_txt,data_db])
data_all = data_all.sort_values(0)
data_all = data_all.reset_index(drop=True)
data_all = data_all.drop(data_all.index[[0]]) 
missing_mp = data_all.drop_duplicates(keep = False, subset = 0)
print(missing_mp)



    

            0         1         2         3         4         5   6
141     978.0  0.192432  0.192432  0.000000  0.192432  0.192432  db
380    2715.0  0.099083  0.275283  0.035726  0.134809  0.311009  db
714    4964.0  0.151526  0.151526  0.000000  0.151526  0.151526  db
1078   7433.0  0.138722  0.138722  0.000000  0.138722  0.138722  db
1447   8716.0  0.086541  0.180582  0.041120  0.127662  0.221702  db
2011  12277.0  0.099811  0.261069  0.044124  0.143936  0.305194  db
2308  15252.0  0.159021  0.159021  0.000000  0.159021  0.159021  db
2335  15642.0  0.125268  0.125268  0.000000  0.125268  0.125268  db
2336  15643.0  0.106370  0.106370  0.000000  0.106370  0.106370  db
2383  16238.0  0.176788  0.176788  0.000000  0.176788  0.176788  db
2490  17173.0  0.142810  0.142810  0.000000  0.142810  0.142810  db
2805  18988.0  0.093902  0.126566  0.050591  0.144493  0.177157  db
2990  20740.0  0.165212  0.165212  0.000000  0.165212  0.165212  db
2997  20809.0  0.086727  0.106920  0.035359  0.1

In [ ]:
mpfile